In [1]:
import os

# import healpy as hp
import numpy as np
import skyproj
import skyproj.hpx_utils
import proplot as pplt
# import matplotlib.pyplot as plt
import fitsio

import rubin_sim
from rubin_sim import maf
from rubin_sim.scheduler.utils import SkyAreaGenerator
# from rubin_sim.utils import raDec2Hpid

from rubin_sim import maf

In [2]:
nside = 128

sm = SkyAreaGenerator(nside=nside)
footprints_hp_array, labels = sm.return_maps()
wfd_indx = np.where((labels == 'lowdust') | (labels == 'virgo'))[0]

In [3]:
import tqdm

roll_uniform_maps_arr = fitsio.read("roll_uniform_early_half_mjdp0_v3.4_10yrs_nside128_bins1000.fits")
baseline_maps_arr = fitsio.read("baseline_v3.4_10yrs_nside128_bins1000.fits")
xy = np.linspace(0, 3653, roll_uniform_maps_arr.shape[0]+1)[1:]/365.25

vmax = max(
    np.median(roll_uniform_maps_arr[-1, wfd_indx]),
    np.median(baseline_maps_arr[-1, wfd_indx]),
)


def _plot_map(marr, ind, ax, title):
    import skyproj
    import skyproj.hpx_utils
    import proplot as pplt

    if ind == -1:
        imap = marr[ind, :].copy() * 0

    else:    
        imap = marr[ind, :].copy()
        msk = imap < 15
        imap[msk] = 0
        imap = imap / vmax * 10
        imap[msk] = -1

    lon, lat, xymap = skyproj.hpx_utils.hpxmap_to_xy(
        imap,
        [0, 360],
        [-90, 90],
        xsize=600,
    )

    cmap = pplt.constructor.Colormap(
        "marine",
        right=0.85,
        left=0.15,
    )
    
    cmap = pplt.constructor.Colormap(
        [(1, 1, 1, 1)] + [
            cmap(x)
            for x in np.linspace(0, 1, 99)
        ]
    )
        
    m = ax.pcolormesh(
        lon, lat, xymap,
        cmap=cmap, 
        vmin=0, 
        vmax=10, 
        discrete=False,
    )
    ax.format(
        title="%s rolling" % title,
    )
    ax.invert_xaxis()
    return m, cmap

def _plot_ind_both(ind, bname):
    import skyproj
    import skyproj.hpx_utils
    import proplot as pplt
    

    fig, axs = pplt.subplots(
        nrows=1, ncols=2,
        proj="moll",
        figsize=(9, 3),
    )
    
    mvals = []
    ax = axs[0, 0]
    mvals.append(_plot_map(baseline_maps_arr, ind, ax, "baseline"))
    ax = axs[0, 1]
    mvals.append(_plot_map(roll_uniform_maps_arr, ind, ax, "uniform/optimized"))
    
    fig.colorbar(
        mvals[0][0], 
        loc="l", 
        title="r+i+z obs. time [LSST one-year equiv.]",
        length=0.8,
        ticks=np.arange(11),
    )
    if ind == -1:
        yr = 0.0
    else:
        yr = xy[ind]
    fig.format(
        suptitle="Rubin LSST WFD Survey at Year %5.2f" % yr
    )
    fig.savefig(bname + "_%04d.png" % (ind+1), dpi=200)
    # return fig

base = "comp_roll"
os.makedirs(base, exist_ok=True)
os.system(f"rm -r ./{base}/*")

# for ind in [0, 39]:
#     fig = _plot_ind_both(ind, f"{base}/{base}")


import joblib

jobs = [
    joblib.delayed(_plot_ind_both)(
        ind, f"{base}/{base}"
    )
    for ind in [-1] + list(range(baseline_maps_arr.shape[0]))
]

with joblib.Parallel(n_jobs=8, backend="loky", verbose=100) as exc:
    exc(jobs)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.


/Users/beckermr/mambaforge/envs/rubin-sims/lib/python3.10/site-packages/cartopy/mpl/geoaxes.py:403: UserWarning: The `map_projection` keyword argument is deprecated, use `projection` to instantiate a GeoAxes instead.
  warnings.warn("The `map_projection` keyword argument is "
/Users/beckermr/mambaforge/envs/rubin-sims/lib/python3.10/site-packages/cartopy/mpl/geoaxes.py:403: UserWarning: The `map_projection` keyword argument is deprecated, use `projection` to instantiate a GeoAxes instead.
  warnings.warn("The `map_projection` keyword argument is "
/Users/beckermr/mambaforge/envs/rubin-sims/lib/python3.10/site-packages/cartopy/mpl/geoaxes.py:403: UserWarning: The `map_projection` keyword argument is deprecated, use `projection` to instantiate a GeoAxes instead.
  warnings.warn("The `map_projection` keyword argument is "
/Users/beckermr/mambaforge/envs/rubin-sims/lib/python3.10/site-packages/cartopy/mpl/geoaxes.py:403: UserWarning: The `map_projection` keyword argument is deprecated, use

[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:   32.8s
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   32.9s


/Users/beckermr/mambaforge/envs/rubin-sims/lib/python3.10/site-packages/cartopy/mpl/geoaxes.py:403: UserWarning: The `map_projection` keyword argument is deprecated, use `projection` to instantiate a GeoAxes instead.
  warnings.warn("The `map_projection` keyword argument is "
/Users/beckermr/mambaforge/envs/rubin-sims/lib/python3.10/site-packages/cartopy/mpl/geoaxes.py:403: UserWarning: The `map_projection` keyword argument is deprecated, use `projection` to instantiate a GeoAxes instead.
  warnings.warn("The `map_projection` keyword argument is "
/Users/beckermr/mambaforge/envs/rubin-sims/lib/python3.10/site-packages/cartopy/mpl/geoaxes.py:403: UserWarning: The `map_projection` keyword argument is deprecated, use `projection` to instantiate a GeoAxes instead.
  warnings.warn("The `map_projection` keyword argument is "
/Users/beckermr/mambaforge/envs/rubin-sims/lib/python3.10/site-packages/cartopy/mpl/geoaxes.py:403: UserWarning: The `map_projection` keyword argument is deprecated, use

[Parallel(n_jobs=8)]: Done   3 tasks      | elapsed:   33.1s


...

[Parallel(n_jobs=8)]: Done 985 tasks      | elapsed: 59.6min


/Users/beckermr/mambaforge/envs/rubin-sims/lib/python3.10/site-packages/cartopy/mpl/geoaxes.py:403: UserWarning: The `map_projection` keyword argument is deprecated, use `projection` to instantiate a GeoAxes instead.
  warnings.warn("The `map_projection` keyword argument is "
/Users/beckermr/mambaforge/envs/rubin-sims/lib/python3.10/site-packages/cartopy/mpl/geoaxes.py:403: UserWarning: The `map_projection` keyword argument is deprecated, use `projection` to instantiate a GeoAxes instead.
  warnings.warn("The `map_projection` keyword argument is "


[Parallel(n_jobs=8)]: Done 986 tasks      | elapsed: 59.6min


/Users/beckermr/mambaforge/envs/rubin-sims/lib/python3.10/site-packages/cartopy/mpl/geoaxes.py:403: UserWarning: The `map_projection` keyword argument is deprecated, use `projection` to instantiate a GeoAxes instead.
  warnings.warn("The `map_projection` keyword argument is "
/Users/beckermr/mambaforge/envs/rubin-sims/lib/python3.10/site-packages/cartopy/mpl/geoaxes.py:403: UserWarning: The `map_projection` keyword argument is deprecated, use `projection` to instantiate a GeoAxes instead.
  warnings.warn("The `map_projection` keyword argument is "
/Users/beckermr/mambaforge/envs/rubin-sims/lib/python3.10/site-packages/cartopy/mpl/geoaxes.py:403: UserWarning: The `map_projection` keyword argument is deprecated, use `projection` to instantiate a GeoAxes instead.
  warnings.warn("The `map_projection` keyword argument is "
/Users/beckermr/mambaforge/envs/rubin-sims/lib/python3.10/site-packages/cartopy/mpl/geoaxes.py:403: UserWarning: The `map_projection` keyword argument is deprecated, use

[Parallel(n_jobs=8)]: Done 997 out of 1001 | elapsed: 60.3min remaining:   14.5s
[Parallel(n_jobs=8)]: Done 1001 out of 1001 | elapsed: 60.4min finished


In [8]:
import os
import moviepy.video.io.ImageSequenceClip

image_files = sorted([
    os.path.join(base,img)
    for img in os.listdir(base)
])

dind = roll_uniform_maps_arr.shape[0] // 10
tails = [
    "_%04d." % (ind * dind)
    for ind in [1, 4, 7, 10]
]
print(tails)

n_extra = dind
seconds_per_year = 4
fps = dind / seconds_per_year
new_image_files = []
for fname in image_files:
    if any(tail in fname for tail in tails):
        new_image_files += [fname] * n_extra
    else:
        new_image_files.append(fname)

clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(new_image_files, fps=fps)
clip.write_videofile(f'{base}.mp4')

['_0100.', '_0400.', '_0700.', '_1000.']
Moviepy - Building video comp_roll.mp4.
Moviepy - Writing video comp_roll.mp4



Moviepy - Done !
Moviepy - video ready comp_roll.mp4
